In [34]:
import numpy as np
import pandas as pd 
import torch

from collections import defaultdict
from importlib import import_module
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
from tqdm import tqdm

from skp.toolbox.functions import load_model_from_config

In [35]:
cfg_name = "mammo.cfg_ddsm_pretrained_aux_losses"
cfg = import_module(f"skp.configs.{cfg_name}").cfg
weights_path = cfg.save_dir + cfg_name + "/ce2bf46d/fold0/checkpoints/last.ckpt"
model = load_model_from_config(cfg, weights_path, device="cuda:0", eval_mode=True)
dataset = import_module(f"skp.datasets.{cfg.dataset}").Dataset(cfg, mode="val")

Enabling gradient checkpointing ...
Loading pretrained backbone from /home/ian/projects/SKP/experiments/mammo/mammo.cfg_ddsm_seg_cls_v02/445e5ce2/fold0/checkpoints/last.ckpt ...
missing keys: ['conv_head.weight', 'bn2.weight', 'bn2.bias', 'bn2.running_mean', 'bn2.running_var']
Loading weights from /home/ian/projects/SKP/experiments/mammo/mammo.cfg_ddsm_pretrained_aux_losses/ce2bf46d/fold0/checkpoints/last.ckpt ...


In [38]:
for v in model.named_buffers():
    print(v)

('backbone.bn1.running_mean', tensor([-0.0252, -1.2663,  1.5484, -0.0055,  0.0395,  1.4614, -0.0218,  1.0093,
         1.0714,  0.1304,  2.5272, -0.4632, -0.1694,  0.0900,  0.0282,  0.0656,
        -0.0126, -2.3625,  1.2331,  0.3791,  0.9984, -1.1975,  0.0731, -0.1500],
       device='cuda:0'))
('backbone.bn1.running_var', tensor([0.2020, 0.8760, 1.2845, 0.2165, 0.2986, 1.1442, 0.2281, 0.5459, 0.6266,
        0.1394, 3.3552, 0.1125, 0.0155, 0.0044, 0.2961, 0.2187, 0.4216, 2.9184,
        0.8151, 0.0788, 0.5399, 0.7674, 0.2949, 0.0126], device='cuda:0'))
('backbone.bn1.num_batches_tracked', tensor(4266, device='cuda:0'))
('backbone.blocks.0.0.bn1.running_mean', tensor([ -3.4717,  -6.1336,  -2.8761,  -3.4592, -10.7691,  -7.6037,  -4.3330,
         -9.2876,  -0.0600,  -4.4574,  -8.6690, -20.2958,  -4.0325,  -3.5185,
         -4.5534,  -1.0941,  -2.9310,  -4.8121, -17.4551,  10.1372,   2.2359,
         -1.4631,  -5.5342,  -5.8302], device='cuda:0'))
('backbone.blocks.0.0.bn1.running_var', 

('backbone.blocks.4.2.bn3.running_mean', tensor([ 0.4241, -0.1171,  0.0243, -0.2313, -0.0254, -0.1832,  0.3368,  0.1206,
         0.1336,  0.0325,  0.0130, -0.0619, -0.0660, -0.1088, -0.0085, -0.3250,
         0.0590, -0.1651, -0.2821,  0.2386, -0.0664,  0.0325, -0.1771, -0.3783,
        -0.6834, -0.1847,  0.2711, -0.1729,  0.3541, -0.3969, -0.5118, -0.0133,
        -0.2263, -0.2745,  0.0565,  0.0719,  0.1068, -0.1293,  0.0367,  0.2108,
         0.4305,  0.1363, -0.2203, -0.1259, -0.0123,  0.0135, -0.3302, -0.1201,
         0.1849,  0.1879, -0.2406,  0.0887, -0.1240, -0.0595, -0.0537,  0.3234,
        -0.7368, -0.1893, -0.2519, -0.0860,  0.0422,  0.0710, -0.1965, -0.0275,
         0.1746, -0.0509, -0.2709,  0.0374, -0.0130,  0.0307,  0.3259,  0.0187,
         0.1552, -0.0687,  0.0597, -0.2401,  0.1905, -0.0528,  0.1400,  0.2437,
         0.0318, -0.2639,  0.0688, -0.0185, -0.1391, -0.1561, -0.2091,  0.1378,
         0.0778, -0.0928, -0.2894,  0.4384, -0.0692, -0.1661, -0.1143, -0.1477,

: 

In [30]:
loader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=13, drop_last=False)

In [31]:
pred_dict = defaultdict(list)
for batch in tqdm(loader, total=len(loader)):
    x = batch["x"]
    x = x.to("cuda:0")
    with torch.inference_mode():
        p = model({"x": x})
    pred_dict["y_pred"].append(p["logits"][:, 0].cpu().numpy())
    pred_dict["y_true"].append(batch["y"][:, 0].numpy())
    pred_dict["group_index"].append(batch["group_index"].numpy())

  0%|          | 0/684 [00:00<?, ?it/s]/home/ian/miniconda3/envs/skp/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/ian/miniconda3/envs/skp/lib/python3.12/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 684/684 [05:02<00:00,  2.26it/s]


In [32]:
pred_df_dict = {key: np.concatenate(val) for key, val in pred_dict.items()}
pred_df = pd.DataFrame(pred_df_dict)
pred_df["y_pred_sigmoid"] = pred_df.y_pred.apply(lambda x: 1 / (1 + np.exp(-x)))
pred_df

,y_pred,y_true,group_index,y_pred_sigmoid
0,-5.989569,0.0,4,0.002498
1,-5.435151,0.0,4,0.004342
2,-6.257034,0.0,5,0.001913
3,-4.784924,0.0,5,0.008286
4,-7.784300,0.0,18,0.000416
...,...,...,...,...
10938,-7.824510,0.0,23811,0.000400
10939,-4.341679,0.0,23818,0.012847
10940,-5.661392,0.0,23818,0.003466
10941,-5.778911,0.0,23819,0.003083


In [33]:
pred_df_grouped_mean = pred_df.groupby("group_index").mean()
print(roc_auc_score(pred_df_grouped_mean.y_true, pred_df_grouped_mean.y_pred))
print(roc_auc_score(pred_df_grouped_mean.y_true, pred_df_grouped_mean.y_pred_sigmoid))

0.9251408742160389
0.9253234005072534


In [28]:
pred_df_grouped_mean = pred_df.groupby("group_index").mean()
print(roc_auc_score(pred_df_grouped_mean.y_true, pred_df_grouped_mean.y_pred))
print(roc_auc_score(pred_df_grouped_mean.y_true, pred_df_grouped_mean.y_pred_sigmoid))

0.9245911729436609
0.9249795719121751
